# Initial Setup.

In [1]:
from ipywidgets import Button, HBox, VBox, Label, BoundedIntText, HTML, Dropdown
import bqplot
import datetime
import dateutil.parser
import ee
import ipywidgets
import ipywidgets as widgets
import IPython.display
import numpy as np
import pprint
import pandas as pd
import traitlets
import ipyleaflet
import geopandas
import bqplot.pyplot as plt
from dateutil import parser

from bqplot.interacts import BrushSelector, BrushIntervalSelector

import sqlite3
import os

import ccd

# Turn off pandas warning
pd.options.mode.chained_assignment = None

# Configure the pretty printing output.
pp = pprint.PrettyPrinter(depth=4)

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
# Authenticate to the Earth Engine servers, and initialize the ee module.
ee.Initialize()

In [4]:
# Define database path and create if necessary
dbPath = '/Users/ericbullock/Google Drive/Work/EarthEngine/referenceCollection/database/database'
if not os.path.isfile(dbPath):
    c.execute('''CREATE TABLE sample
             (id real, lat real, lon real, year1 real, year2 real, class text, class2 text, confidence real)''')
    print('Database created')


In [5]:
# Functions to load collections, merge, filter and calculate statistics

# Filter collection by point and date
def collection_filtering(point, collection_name, year_range, doy_range):
    collection = ee.ImageCollection(collection_name)\
    .filterBounds(point)\
    .filter(ee.Filter.calendarRange(year_range[0], year_range[1], 'year'))\
    .filter(ee.Filter.dayOfYear(doy_range[0],doy_range[1]))
    return collection

# Landsat stack renamers, C1
def stack_renamer_l4_7_C1(img):
    band_list = ['B1', 'B2','B3','B4','B5','B7', 'B6','pixel_qa']
    name_list = ['BLUE','GREEN','RED','NIR','SWIR1','SWIR2','THERMAL','pixel_qa']
    return ee.Image(img).select(band_list).rename(name_list)

def stack_renamer_l8_C1(img):
    band_list = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10','pixel_qa'];
    name_list = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2','THERMAL','pixel_qa'];
    return ee.Image(img).select(band_list).rename(name_list);


# Cloud masking for C1, L4-L7. Operators capitalized to
# avoid confusing with internal Python operators
def cloud_mask_l4_7_C1(img):
    pqa = ee.Image(img).select(['pixel_qa'])
    mask = (pqa.eq(66)).Or(pqa.eq(130))\
    .Or(pqa.eq(68)).Or(pqa.eq(132))
    return ee.Image(img).updateMask(mask)
  
# Cloud masking for C1, L8
def cloud_mask_l8_C1(img):
    pqa = ee.Image(img).select(['pixel_qa'])
    mask = (pqa.eq(322)).Or(pqa.eq(386)).Or(pqa.eq(324))\
    .Or(pqa.eq(388)).Or(pqa.eq(836)).Or(pqa.eq(900))
    return ee.Image(img).updateMask(mask)


# Calculate period statistics, seems to work but it's so ugly!
def calc_period_stats(yr_list, collection, reducer, stat_name, bands):
    def auxfnc(year):
        start = ee.Date(year)
        end = start.advance(step, 'year').advance(-1, 'day')

        return collection\
        .select(bands)\
        .filterDate(start, end)\
        .reduce(reducer)\
        .set({'system:time_start': start, 'system:time_end': end, 'statistic': stat_name})
    period_stats = yr_list.map(auxfnc)
    return period_stats

def fc2df(fc):
    # Convert a FeatureCollection into a pandas DataFrame
    # Features is a list of dict with the output
    features = fc.getInfo()['features']
    dictarr = []
    for f in features:
        # Store all attributes in a dict
        attr = f['properties']
        dictarr.append(attr)

    return pd.DataFrame(dictarr)

def fc2dfgeo(fc):
    # Convert a FeatureCollection into a pandas DataFrame
    # Features is a list of dict with the output
    features = fc.getInfo()['features']

    dictarr = []

    for f in features:
        # Store all attributes in a dict
        attr = f['properties']
        # and treat geometry separately
        attr['geometry'] = f['geometry']
        dictarr.append(attr)

    df = geopandas.GeoDataFrame(dictarr)
    return df

def get_df(collection, coords, band):
    point = ee.Geometry.Point(coords)
    # Sample for a time series of values at the point.
    geom_values = collection.filterBounds(point).select(band).getRegion(geometry=point, scale=30)
    geom_values_list = ee.List(geom_values).getInfo()
    # Convert to a Pandas DataFrame.
    header = geom_values_list[0]
    data = pd.DataFrame(geom_values_list[1:], columns=header)
    data['datetime'] = pd.to_datetime(data['time'], unit='ms', utc=True)
    data.set_index('time')
    data = data.sort_values('datetime')
    data = data[['id', 'datetime', band]]
    return data

def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

In [6]:
# Load, filter and merge collections

def get_full_collection(coords, year_range, doy_range):
    point = ee.Geometry.Point(coords)
    l8_renamed = collection_filtering(point, 'LANDSAT/LC08/C01/T1_SR', year_range, doy_range)\
        .map(stack_renamer_l8_C1)
    l8_filtered1 = l8_renamed.map(cloud_mask_l8_C1)

    l7_renamed = collection_filtering(point, 'LANDSAT/LE07/C01/T1_SR', year_range, doy_range)\
        .map(stack_renamer_l4_7_C1);
    l7_filtered1 = l7_renamed.map(cloud_mask_l4_7_C1)

    l5_renamed = collection_filtering(point, 'LANDSAT/LT05/C01/T1_SR', year_range, doy_range)\
        .map(stack_renamer_l4_7_C1)
    l5_filtered1 = l5_renamed.map(cloud_mask_l4_7_C1)


    all_scenes = ee.ImageCollection((l8_filtered1.merge(l7_filtered1)).merge(l5_filtered1)).sort('system:time_start')
    return all_scenes

In [7]:
# PyCCD Functions
# Functions for running pyccd
def make_df_pyccd(collection, point):
    band_list = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2','THERMAL', 'pixel_qa']
    rename_list = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2','THERMAL', 'pixel_qa']
    info = collection.getRegion(point, 30).getInfo()
    header = info[0]
    data = np.array(info[1:])
    iTime = header.index('time')
    time = [datetime.datetime.fromtimestamp(i/1000) for i in (data[0:,iTime].astype(int))]
    time_new = [t.toordinal() for t in (time)]
    iBands = [header.index(b) for b in band_list]
    yData = data[0:,iBands].astype(np.float)
    df = pd.DataFrame(data=yData, index=list(range(len(yData[:,0]))), columns=rename_list)
    df['time'] = time_new
    return df

def update_df(df, df2):
    df = df.append(df2)
    return df

def plot_pyccd(results, df, band, plotband, dates, yl, ylabel):
    mask = np.array(results['processing_mask']).astype(np.bool_)
    predicted_values = []
    prediction_dates = []
    break_dates = []
    start_dates = []

    #print(results)
    for num, result in enumerate(results['change_models']):
        days = np.arange(result['start_day'], result['end_day'] + 1)
        prediction_dates.append(days)
        break_dates.append(result['break_day'])
        start_dates.append(result['start_day'])
        intercept = result[list(result.keys())[6+band]]['intercept']
        coef = result[list(result.keys())[6+band]]['coefficients']
    
        predicted_values.append(intercept + coef[0] * days +
                                coef[1]*np.cos(days*1*2*np.pi/365.25) + coef[2]*np.sin(days*1*2*np.pi/365.25) +
                                coef[3]*np.cos(days*2*2*np.pi/365.25) + coef[4]*np.sin(days*2*2*np.pi/365.25) +
                                coef[5]*np.cos(days*3*2*np.pi/365.25) + coef[6]*np.sin(days*3*2*np.pi/365.25))
    
    num_breaks = len(break_dates)

    # eric here
    break_y = [plotband[dates == i][0] for i in break_dates]

    #break_y = [0] * num_breaks
    break_dates_plot = [datetime.datetime.fromordinal(i).strftime('%Y-%m-%d %H:%M:%S.%f') for i in break_dates]

    plot_dates = np.array([datetime.datetime.fromordinal(i) for i in (dates)])

    # Predicted curves
    all_dates = []
    all_preds = []
    for _preddate, _predvalue in zip(prediction_dates, predicted_values):
        all_dates.append(_preddate)
        all_preds.append(_predvalue)
        
    all_preds = [item for sublist in all_preds for item in sublist]
    all_dates = [item for sublist in all_dates for item in sublist]

    date_ord = [datetime.datetime.fromordinal(i).strftime('%Y-%m-%d %H:%M:%S.%f') for i in all_dates]
    _x = np.array(date_ord, dtype='datetime64')
    _y = all_preds
    lc4.x = _x
    lc4.y = _y
    lc5.x = np.array(break_dates_plot, dtype='datetime64')
    lc5.y = break_y
    
def get_full_collection(coords, year_range, doy_range):
    point = ee.Geometry.Point(coords)
    l8_renamed = collection_filtering(point, 'LANDSAT/LC08/C01/T1_SR', year_range, doy_range)\
        .map(stack_renamer_l8_C1)
    l8_filtered1 = l8_renamed.map(cloud_mask_l8_C1)

    l7_renamed = collection_filtering(point, 'LANDSAT/LE07/C01/T1_SR', year_range, doy_range)\
        .map(stack_renamer_l4_7_C1);
    l7_filtered1 = l7_renamed.map(cloud_mask_l4_7_C1)

    l5_renamed = collection_filtering(point, 'LANDSAT/LT05/C01/T1_SR', year_range, doy_range)\
        .map(stack_renamer_l4_7_C1)
    l5_filtered1 = l5_renamed.map(cloud_mask_l4_7_C1)


    all_scenes = ee.ImageCollection((l8_filtered1.merge(l7_filtered1)).merge(l5_filtered1)).sort('system:time_start')
    return all_scenes

In [8]:
class interface(object):
    """A class to hold interface for defining sample attributes"""
    
    def __init__(self, path):
        interface.pathDB = path

    # Years slider
    years = widgets.IntRangeSlider(
            value=[2012, 2015],
            min=1990,
            max=2018,
            step=1,
            description='Years:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d',
        )
    
    # Secondary class information
    tier_drop = widgets.Dropdown(
                options=['Secondary Class Information'],
                value='Secondary Class Information',
                #rows=10,
                description='Secondary:',
                disabled=False
            )
    
    def lc_clicked(_, old_selection, selection):
        """ Generate secondary class selector after initial class is chosen """
        if selection == 'Forest':
            interface.tier_drop.set_trait('options', ['Forest Type','Deciduous', 'Coniferous', 'Mangrove','Forested Wetland'])
        elif selection == 'Water':
            interface.tier_drop.set_trait('options', ['Water Type','Type1', 'Type2', 'Type3','Type4'])
        elif selection == 'Developed':
            interface.tier_drop.set_trait('options', ['Developed Type','Type1', 'Type2', 'Type3','Type4'])
        elif selection == 'Ag/Past':
            interface.tier_drop.set_trait('options', ['Ag/Pas Type','Type1', 'Type2', 'Type3','Type4'])
    
    # Primary class selector
    lc = widgets.Dropdown(
                options=['Select Class','Forest', 'Developed', 'Water','Ag/Past'],
                value='Select Class',
                #rows=10,
                description='Select Class',
                disabled=False
              )

    # When selected, run lc_clicked function
    lc.on_trait_change(lc_clicked, 'value')
    
    # Display current validity of sample
    valid = widgets.Valid(
                value=False,
                description='',
            )

    # Create button for checking validity
    validate = widgets.Button(
                value=False,
                description='Validate',
                disabled=False,
                button_style='', 
                icon='check'
                )
    
    def check_val_status(selection):
        """ Check the validity of the current sample and change valid widget accordingly """
        selected_secondary_lc = False
        wrote_correct_lc = False
        if interface.tier_drop.value != 'Secondary Class Information':
            selected_secondary_lc = True
        else:
            print("Must specify secondary class information!")
        if interface.lc.value.capitalize() == interface.textClass.value.capitalize():
            wrote_correct_lc = True
        if selected_secondary_lc and wrote_correct_lc:
            interface.valid.value = True
            interface.save_button.disabled = False
            #interface.valid.set_trait('value',True)
            #interface.save_button.set_trait('disabled',False)

    # Call function check_val_status when button is clicked
    validate.on_click(check_val_status)
    
    # Save the current sample, only possible when validity has been approved
    save_button = widgets.Button(
                value=False,
                description='Save',
                disabled=True,
                button_style='', 
                )
    
    def save_sample(selection):
        """ Save the sample to the database """
        # TODO: This should be backed up somehow
        
        # Connect to the database
        conn = sqlite3.connect(interface.pathDB)

        c = conn.cursor()

        # Get everything in right format
        year1 = interface.years.value[0]
        year2 = interface.years.value[1]
        lcClass = interface.lc.value
        class2 = interface.tier_drop.value
        conf = interface.confidence.value

        # TODO: Read ID and geographic information from sample
        idSample = 1
        lat = 1
        lon = 1

        sampleInput = (idSample, lat, lon, year1, year2, lcClass, class2, conf)

        # Put sample information into database
        c.execute("""insert into samples
                  values {i}""".format(i=sampleInput))

        # Save (commit) the changes
        conn.commit()

        # Close the cursor
        c.close()

        # Reset the buttons
        #years.set_trait('value',[2012, 2015])
        interface.lc.set_trait('value', 'Select Class')
        interface.valid.set_trait('value',False)
        interface.save_button.set_trait('disabled',True)
        interface.textClass.set_trait('value','')

    # Call function save_sample when clicking save button
    save_button.on_click(save_sample)
    
    # Manually input class label
    textClass = widgets.Text(
                    value='',
                    placeholder='',
                    description='Enter Class',
                    disabled=False
                )
    
    # Interpreters confidence
    confidence = widgets.IntSlider(
                    value=0,
                    min=0,
                    max=3,
                    step=1,
                    description='Confidence:',
                    disabled=False,
                    continuous_update=False,
                    orientation='horizontal',
                    readout=True,
                    readout_format='d'
                    
                )
    
    # Notes
    notes = widgets.Textarea(
                value='Enter any useful or interesting information about the sample.',
                placeholder='Enter any useful or interesting information about the sample',
                description='Notes:',
                layout=widgets.Layout(width='70%'),
                disabled=False
            )
    


/Users/ericbullock/anaconda/envs/measures/lib/python3.6/site-packages/ipykernel_launcher.py:51: DeprecationWarning: on_trait_change is deprecated in traitlets 4.1: use observe instead


In [9]:
# Set global variables
year_range = [ 2000, 2018 ];
doy_range = [ 1, 365 ];
step = 1 #in years

In [10]:
# Load feature collection
testfeat = ee.FeatureCollection("users/parevalo_bu/sample_13_15_labels_pts")

# Get collection as geopandas df
df = fc2dfgeo(testfeat)

In [11]:
# interface stuff
# Generate sample interface
newInterface = interface(dbPath)
newInterface.valid.value = False

# Create widget controls
current_id = 0
next_pt = Button(value=False, description='Next point', disabled=False) 
previous_pt = Button(value=False, description='Previous point', disabled=False)
pyccd_button = Button(value=False, description='Run PyCCD', disabled=False)
band_selector = Dropdown(options=['BLUE', 'GREEN', 'RED','NIR','SWIR1','SWIR2'], description='Select band', value=None)
coords_label = Label()
pt_message = HTML()
time_label = HTML(value='')
selected_label = HTML("ID of selected point")
hover_label = HTML("test value")


# Plot scales.
lc1_x = bqplot.DateScale(min=datetime.date(2001, 2, 1), max=datetime.date(2018, 1, 1))
lc2_y = bqplot.LinearScale(min=0, max=4000)

# Main scatter plot for samples
lc2 = bqplot.Scatter(
    x=[],
    y=[],
    scales={'x': lc1_x, 'y': lc2_y},
    size=[1,1],
    interactions={'click': 'select', 'hover': 'tooltip'},
    selected_style={'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'},
    unselected_style={'opacity': 0.5},
    display_legend=True,
    labels = ['Sample point']
)

# Scatter plot for clicked points in map
lc3 = bqplot.Scatter(
    x=[],
    y=[],
    scales={'x': lc1_x, 'y': lc2_y},
    size=[1,1],
    colors=['red'],
    interactions={'click': 'select', 'hover': 'tooltip'},
    selected_style={'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'},
    unselected_style={'opacity': 0.5},
    display_legend=True,
    labels = ['Clicked point']
)

lc4 = bqplot.Lines(
    x=[],
    y=[],
    colors=['black'],
    stroke_width=3,
    scales={'x': lc1_x, 'y': lc2_y},
    size=[1,1],
)

lc5 = bqplot.Scatter(
    x=[],
    y=[],
    marker='triangle-up',
    colors=['red'],
    scales={'x': lc1_x, 'y': lc2_y},
    size=[1,1],
    display_legend=False,
    labels = ['Model Endpoint']
)

# Plot axes.
x_ax1 = bqplot.Axis(label='Date', scale=lc1_x, num_ticks = 6, tick_format='%Y')
x_ay2 = bqplot.Axis(label='SWIR1', scale=lc2_y, orientation='vertical')

# Brush selector for figure
brushintsel = BrushIntervalSelector(scale=lc1_x)

# Create a figure.
fig = bqplot.Figure(
    marks=[lc2, lc3, lc4, lc5],
    axes=[x_ax1, x_ay2],
    layout={'height':'300px', 'width':'800px'},
    #interaction=brushintsel
)

# Get years from brush selector
def get_scatter_years(*args):
    date1 = parser.parse(str(brushintsel.selected[0])).year
    date2 = parser.parse(str(brushintsel.selected[1])).year
    newInterface.years = [date1, date2]

    text_brush.value = "Selected year range: {a}-{b}".format(a=date1, b=date2)

# The following text widgets are used to display the `selected` attributes
text_brush = HTML(value = 'Selected year range:')

brushintsel.observe(get_scatter_years, 'selected')

#display(text_brush)
#get_scatter_years()

# Go back or forth between sample points
def advance(b):
    # Plot point in map
    global current_id
    lc4.x = []
    lc4.y = []
    lc5.x = []
    lc5.y = []
    lc5.display_legend=False
    global pyccd_flag
    pyccd_flag = False
    newInterface.valid.value = False
    newInterface.save_button.disabled = True
    current_id += 1
    gjson = ipyleaflet.GeoJSON(data=df['geometry'][current_id])
    m.center = gjson.data['coordinates'][::-1]
    m.zoom = 12
    m.add_layer(gjson)
    pt_message.value = "Point ID: {}".format(current_id)
    # Plot TS
    current_band = band_list[band_index]
    point_ts = get_full_collection(df['geometry'][current_id]['coordinates'], year_range, doy_range)
    point_df = get_df(point_ts, df['geometry'][current_id]['coordinates'], current_band).dropna()
    lc2.x = point_df['datetime']
    lc2.y = point_df[current_band]
    

def decrease(b):
    # Plot point in map
    global current_id
    lc4.x = []
    lc4.y = []
    lc5.x = []
    lc5.y = []
    lc5.display_legend=False
    global pyccd_flag
    pyccd_flag = False
    newInterface.valid.value = False
    newInterface.save_button.disabled = True
    current_id -= 1
    gjson = ipyleaflet.GeoJSON(data=df['geometry'][current_id])
    m.center = gjson.data['coordinates'][::-1]
    m.zoom = 12
    m.add_layer(gjson)
    pt_message.value = "Point ID: {}".format(current_id)
    # Plot TS
    current_band = band_list[band_index]
    point_ts = get_full_collection(df['geometry'][current_id]['coordinates'], year_range, doy_range)
    point_df = get_df(point_ts, df['geometry'][current_id]['coordinates'], current_band).dropna()
    lc2.x = point_df['datetime']
    lc2.y = point_df[current_band]
    
# Create map interaction handler
def handle_interaction(**kwargs):
    if kwargs.get('type') == 'mousemove':
        coords_label.value = str(kwargs.get('coordinates'))

# Show selected point ID
def click_event(self, target):
    global current_id
    pt_index = target['data']['index']
    #selected_label.value = str(pt_index)
    # Plot TS
    gjson = ipyleaflet.GeoJSON(data=df['geometry'][current_id])
    point_ts = get_full_collection(df['geometry'][current_id]['coordinates'], year_range, doy_range)
    point_df = get_df(point_ts, df['geometry'][current_id]['coordinates'], 'SWIR1').dropna()
    # Find clicked image. .values needed to access the nth element of that list instead of indexing by ID
    image_id = point_df['id'].values[pt_index]
    selected_image = ee.Image(point_ts.filterMetadata('system:index', 'equals', image_id).first())
    tile_url = GetTileLayerUrl(selected_image.visualize(min=0, max=6000, bands= ['NIR', 'SWIR1', 'RED']))
    m.add_layer(ipyleaflet.TileLayer(url=tile_url))
    selected_label.value = image_id
    
# Run pyccd
def run_pyccd(b):
    # Run pyCCD on current point
    # Display the legend
    lc5.display_legend=True
    
    # Set flag
    global pyccd_flag
    pyccd_flag = True
    
    # Plot TS
    name_list = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2','THERMAL', 'pixel_qa']
    point_ts = get_full_collection(df['geometry'][current_id]['coordinates'], year_range, doy_range).select(name_list)
 
    # Do PyCCD
    pointGeo = ee.Geometry.Point(df['geometry'][current_id]['coordinates'])
    dfPyCCD = make_df_pyccd(point_ts, pointGeo)
    
    dfPyCCD = dfPyCCD.sort_values('time')

    dfPyCCD['pixel_qa'][dfPyCCD['pixel_qa'].isnull()] = 4
    dfPyCCD[df.isnull()] = 0
    dfPyCCD['pixel_qa'][dfPyCCD['pixel_qa'] > 4] = 0

    #TODO: Paramaterize everything
    params = {'QA_BITPACKED': False,
              'QA_FILL': 255,
              'QA_CLEAR': 0,
              'QA_WATER': 1,
              'QA_SHADOW': 2,
              'QA_SNOW': 3,
              'QA_CLOUD': 4}

    dates = np.array(dfPyCCD['time'])
    blues = np.array(dfPyCCD['BLUE'])
    greens = np.array(dfPyCCD['GREEN'])
    reds = np.array(dfPyCCD['RED'])
    nirs = np.array(dfPyCCD['NIR'])
    swir1s = np.array(dfPyCCD['SWIR1'])
    swir2s = np.array(dfPyCCD['SWIR2'])
    thermals = np.array(dfPyCCD['THERMAL'])
    qas = np.array(dfPyCCD['pixel_qa'])
    results = ccd.detect(dates, blues, greens, reds, nirs, swir1s, swir2s, thermals, qas, params=params)

    band_names = ['Blue SR', 'Green SR', 'Red SR', 'NIR SR', 'SWIR1 SR', 'SWIR2 SR','THERMAL']
    plotlabel = band_names[band_index] 

    plot_arrays = [blues, greens, reds, nirs, swir1s, swir2s]
    plotband = plot_arrays[band_index]
    plot_pyccd(results, df, band_index, plotband, dates, (0, 4000), 'PyCCD Results')
    
    
# Handle hover on marker
def hover_event(self, target):
    hover_label.value = str(target['data']['x'])
    
# Handle marker creation
def handle_draw(self, action, geo_json):
    # Get the selected coordinates from the map's drawing control.
    coords = geo_json['geometry']['coordinates']    
    point_ts = get_full_collection(coords, year_range, doy_range)
    point_df = get_df(point_ts, coords, band_selector.value).dropna()
    lc3.x = point_df['datetime']
    lc3.y = point_df[band_selector.value]

# Update on dropdow selection
def on_band_selection(change):
    new_band = change['new']
    global band_index
    band_index = change['owner'].index

    global current_id
    gjson = ipyleaflet.GeoJSON(data=df['geometry'][current_id])
    point_ts = get_full_collection(df['geometry'][current_id]['coordinates'], year_range, doy_range)
    point_df = get_df(point_ts, df['geometry'][current_id]['coordinates'], new_band).dropna()
    lc2.x = point_df['datetime']
    lc2.y = point_df[new_band]
    x_ay2.label = new_band
    lc4.x = []
    lc4.y = []
    lc5.x = []
    lc5.y = []
    # TODO: This should not have to be ran again
    if pyccd_flag:
        run_pyccd(0)
    
    
# Button and map interactions
next_pt.on_click(advance)
previous_pt.on_click(decrease)
pyccd_button.on_click(run_pyccd)

lc2.on_element_click(click_event)
lc2.tooltip=hover_label
lc2.on_hover(hover_event)

lc3.on_element_click(click_event)
lc3.tooltip=hover_label
lc3.on_hover(hover_event)

band_list =['BLUE', 'GREEN', 'RED','NIR','SWIR1','SWIR2']

band_selector.observe(on_band_selection, names='value')
global band_index
band_index = 4
global pyccd_flag
pyccd_flag = False


# Set up map and widgets, plot!
display(coords_label)
m = ipyleaflet.Map(zoom=5, layout={'height':'400px'}, 
                      center=(3.3890701010382958, -67.32297252983098),dragging=True,
                     close_popup_on_click=False, basemap=ipyleaflet.basemaps.Esri.WorldImagery)

dc = ipyleaflet.DrawControl(marker={'shapeOptions': {'fillcolor': '#ff0000', 'color': '#ff0000'}}, 
                            polygon={}, circle={}, circlemarker={}, polyline={})
dc.on_draw(handle_draw)
m.add_control(dc)
m.on_interaction(handle_interaction)

# Show widgets!
info = HBox([pt_message, time_label], layout=widgets.Layout(width='100%'))
buttons = HBox([previous_pt, next_pt, pyccd_button], layout=widgets.Layout(width='100%'))
panel1 = VBox([info, buttons], layout=widgets.Layout(width='100%'))
panel2 = VBox([band_selector, fig])
controls = VBox([panel1])
display(m)
display(controls)



# Organize into rows and columns with VBox and HBox
left_box = VBox([newInterface.years, newInterface.lc, newInterface.confidence, newInterface.tier_drop, newInterface.textClass])
#right_box = VBox([newInterface.confidence])
save_box = HBox([newInterface.save_button, newInterface.validate, newInterface.valid])
bottom_box = VBox([newInterface.notes, save_box])
display(HBox([left_box]))
#display(HBox([left_box, right_box]), layout=widgets.Layout(flex='1 1 auto'))

display(bottom_box)
display(panel2)

#display(HTML('<style>.container { width:100% !important; }</style>'))

Label(value='')

Map(basemap={'url': 'http://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/…

- Note: Change in dropdown doesn't change clicked TS, needs to be reclicked after the change

In [14]:
IPython.display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [49]:
IPython.display.HTML('''<script>
code_show_err=true; 
function code_toggle_err() {
 if (code_show_err){
 $('div.output_stderr').hide();
 } else {
 $('div.output_stderr').show();
 }
 code_show_err = !code_show_err
} 
$( document ).ready(code_toggle_err);
</script>
To toggle on/off warnings, click <a href="javascript:code_toggle_err()">here</a>.''')

In [111]:
from dateutil import parser

parser.parse('2004-04-28T13:32:01.716000').year

2004